In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from scipy.ndimage import gaussian_filter
import tqdm
from joblib import Parallel, delayed
from markov_functions import *
from itertools import product
import warnings


In [2]:
pa_pitches_filename = './data/paired_filtered.csv'  # or provide a full path

if not os.path.exists(pa_pitches_filename):
    csv_files = [f for f in os.listdir('.') if f.lower().endswith('.csv')]
    if not csv_files:
        raise FileNotFoundError(f"{pa_pitches_filename!r} not found and no CSV files in the current directory.")
    filename = csv_files[0]
    print(f"No file named 'data.csv' found. Using first CSV in cwd: {filename}")



pas = pd.read_csv(pa_pitches_filename)
pas.drop(['pa_seq', 'bases', 'velocities_events', 'pitchCodes_events'], axis=1, inplace=True)
print(f"Loaded {len(pas)} rows and {len(pas.columns)} columns from {pa_pitches_filename}")
# pas.head()


Loaded 27600 rows and 13 columns from ./data/paired_filtered.csv


In [15]:
pas[pas['catcherName']=='宋嘉翔'].tail(50)

,batterName,batterHand,pitcherName,pitcherHand,catcherName,pitchCodes,result,hardness,PAId,pitchTypes_events,coords_events,ball_strike_counts,n_pitches
21730,岳政華,L,陳柏豪,R,宋嘉翔,"['B', 'B', 'B', 'B']",uBB,NaN,356_28,"['FF', 'FF', 'SL', 'FF']","[(1.8444844999999999, -16.2460305), (23.263733...","['0-0', '1-0', '2-0', '3-0']",4.0
21731,陳子豪,L,陳柏豪,R,宋嘉翔,"['B', 'F', 'H']",2B,H,356_29,"['SL', 'SL', 'SL']","[(22.9349935, -0.759841), (-4.086606000000001,...","['0-0', '1-0', '1-1']",3.0
21732,許基宏,L,陳柏豪,R,宋嘉翔,"['B', 'B', 'S', 'H']",GO,H,356_30,"['CH', 'CH', 'SL', 'FF']","[(-10.346436, -32.393636), (-0.460628999999999...","['0-0', '1-0', '2-0', '2-1']",4.0
21733,王威晨,L,陳柏豪,R,宋嘉翔,[],IBB,NaN,50_75,"['FF', 'FF', 'FF', 'FF', 'FF']","[(-0.43110149999999997, -30.558994000000002), ...","['0-0', '1-0', '1-1', '1-2', '1-2']",5.0
21734,曾頌恩,R,陳柏豪,R,宋嘉翔,"['B', 'B', 'H']",FO,M,356_32,"['SL', 'CU', 'SL']","[(-37.039296, -28.767659), (-24.1869595, -38.6...","['0-0', '1-0', '2-0']",3.0
21735,岳東華,L,陳柏豪,R,宋嘉翔,"['B', 'B', 'B', 'B']",uBB,NaN,356_33,"['FF', 'SL', 'SL', 'FF']","[(19.6397245, 9.456674), (11.0728125, 15.38776...","['0-0', '1-0', '2-0', '3-0']",4.0
21736,高宇杰,R,陳柏豪,R,宋嘉翔,"['B', 'H']",FO,H,356_34,"['FF', 'FF']","[(-16.608234500000002, -12.2932825), (-6.51376...","['0-0', '1-0']",2.0
21941,張志豪,L,陳克羿,R,宋嘉翔,['H'],1B,H,359_0,['FF'],"[(8.846439, 6.9921120000000005)]",['0-0'],1.0
21942,岳政華,L,陳克羿,R,宋嘉翔,['BUNT'],SH,S,359_1,['FF'],"[(-7.4862055000000005, 0.9370059999999999)]",['0-0'],1.0
21943,陳子豪,L,陳克羿,R,宋嘉翔,"['S', 'B', 'B', 'H']",GO,M,359_2,"['SL', 'SL', 'FF', 'FF']","[(11.944858, -7.2578594999999995), (-0.4468495...","['0-0', '0-1', '1-1', '2-1']",4.0
